<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [14]:
data=pd.read_csv('/datasets/Churn.csv')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Целевой признак 'Exited',именно его нам необходимо спрогнозировать

In [16]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Уберем столбцы, которые не пригодятся в предсказании целевого признака

In [17]:
data.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

Видно , что в данных присутствуют категориальные признаки, чтобы воспользоваться методами машинного обучения необходимо преобразовать их в численные

In [18]:

gender_one_hot = pd.get_dummies(data["Gender"], drop_first=True)
country_one_hot = pd.get_dummies(data["Geography"], drop_first=True)
data.drop(["Gender", "Geography"], axis=1, inplace=True)
data_ohe = pd.concat([data, gender_one_hot, country_one_hot], axis=1)


In [19]:
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1


Найдем количество пропущенных значений

In [20]:
data_ohe.isna().sum()

CreditScore          0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Male                 0
Germany              0
Spain                0
dtype: int64

In [21]:
target=data['Exited']#отделили целевой признак
features=data_ohe.drop('Exited',axis=1)
features.fillna(1,inplace=True)
features_train, features_test0, target_train, target_test0 = train_test_split(features, target, 
                                                                              test_size=0.2, 
                                                                              random_state=1)
features_valid, features_test, target_valid, target_test = train_test_split(features_test0, target_test0, 
                                                                              test_size=0.5, 
                                                                              random_state=1)



Разбили данные на три выборки: валидационную, обучающую и тестовую

In [22]:
pd.options.mode.chained_assignment=None
numeric=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
scaler=StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric]=scaler.transform(features_train[numeric])
features_valid[numeric]=scaler.transform(features_valid[numeric])
features_test[numeric]=scaler.transform(features_test[numeric])

Подготовка даннных на этом завершена: выборка разбита на две, признаки масштабированы, все признаки приведены к численным , выделили целевой признак, все пропущенные значение удалены. Теперь можно приступать к обучению моделей

## Исследование задачи

Обучим модели машинного обучения без учета балансов классов и измерим значение F1-меры и AUC-ROC

Решающее дерево, лучший результат достигается при глубине в 9 деревьев,F1=0.63,AUC-ROC=0.84

In [23]:

score_ds=0
depth_ds=0
for i in range(1,10):
    DS = DecisionTreeClassifier(max_depth=i,random_state=12345)
    DS.fit(features_train,target_train)
    predictions_ds=DS.predict(features_valid)
    score_ds=f1_score(target_valid, predictions_ds)
    prediction_auc_ds=DS.predict_proba(features_valid)[:,1]
    auc_roc_ds=roc_auc_score(target_valid,prediction_auc_ds)
    print("Глубина дерева:",i)
    print("AUC-ROC:",auc_roc_ds)
    print("F1:",score_ds)

Глубина дерева: 1
AUC-ROC: 0.7197347559523405
F1: 0.0
Глубина дерева: 2
AUC-ROC: 0.7713307793330113
F1: 0.5337423312883436
Глубина дерева: 3
AUC-ROC: 0.8228995842377806
F1: 0.43697478991596644
Глубина дерева: 4
AUC-ROC: 0.8468722568879756
F1: 0.536082474226804
Глубина дерева: 5
AUC-ROC: 0.8581713266965616
F1: 0.6092307692307692
Глубина дерева: 6
AUC-ROC: 0.8563238726447512
F1: 0.5993265993265994
Глубина дерева: 7
AUC-ROC: 0.8573887955143953
F1: 0.603658536585366
Глубина дерева: 8
AUC-ROC: 0.8497063806912132
F1: 0.620253164556962
Глубина дерева: 9
AUC-ROC: 0.8456134023775338
F1: 0.6358024691358024


Случайный лес, лучший результат F1=0.58.

In [24]:
rfc = RandomForestClassifier()
param_grid = { 
    'n_estimators': [10, 20, 30, 40, 50],
    'max_features': ['log2'],
    'max_depth' : [5,7,9,11,13,15],
    'criterion' : ['gini']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_train, target_train)
predictions = CV_rfc.best_estimator_.predict(features_valid)
print("F1:", f1_score(predictions, target_valid))

F1: 0.6035087719298246


У логистической регрессии наблюдается самый низкий результат f1 метрики, всего 0.34

In [25]:
LG = LogisticRegression(max_iter=10000,random_state=12345,solver='liblinear')
LG.fit(features_train,target_train)
predictions_lg=LG.predict(features_valid)
score_lg=f1_score(target_valid, predictions_lg)
prediction_auc_lg=LG.predict_proba(features_valid)[:,1]
auc_roc_lg=roc_auc_score(target_valid,prediction_auc_lg)

print("F1:",score_lg)
print('AUC-ROC:',auc_roc_lg)

F1: 0.3416666666666667
AUC-ROC: 0.7924455120135547


Изучим баланс классов

Доля положительных

In [26]:
sum(data["Exited"]/len(data))

0.20369999999999389

Доля отрицатльных

In [27]:
1-sum(data["Exited"]/len(data))

0.7963000000000061

Были обучены три разные модели: решающее дерево, логистическая регрессия и случайный лес. Лучший результат выдало решающее дерево с глубиной 9,F1=0.63,AUC-ROC=0.84. В наших данных наблюдается дисбаланс с перевесом в сторону отрицательного класса, это влияет на качество обучение моделей, поэтому в следующем пункте будем бороться с дисбалансом.

## Борьба с дисбалансом

Проведем балансировку классов, начнем с метода downsampling,то есть увеличим число меньшего класса

In [28]:
def upsample(features, target, repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, repeat=4)

Показатели у логистической регрессии заметно выросли и F1 достигла значения 0.49

In [29]:
LG = LogisticRegression(max_iter=100000,random_state=12345,solver='liblinear')
LG.fit(features_upsampled_train,target_upsampled_train)
predictions_lg=LG.predict(features_valid)
score_lg=f1_score(target_valid, predictions_lg)
prediction_auc_lg=LG.predict_proba(features_valid)[:,1]
auc_roc_lg=roc_auc_score(target_valid,prediction_auc_lg)
print("F1:",score_lg)
print('AUC-ROC:',auc_roc_lg)

F1: 0.49717514124293793
AUC-ROC: 0.7976442409107302


В моедли случайного леса также наблюдается заметное улучшение, лучшее значение F1=0.6 получается у модели с 8 деревьями

In [30]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_upsampled_train, target_upsampled_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [5, 7, 9, 11, 13, 15],
                         'max_features': ['log2'],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [31]:

predictions = CV_rfc.best_estimator_.predict(features_valid)

print("F1:", f1_score(predictions, target_valid))

F1: 0.6094674556213018


Модель решающего дерева  показывает  снижение качества, F1 мера достигла 0.55 с глубиной в 6 деревьев

In [32]:
score_ds=0
depth_ds=0
for i in range(1,10):
    DS = DecisionTreeClassifier(max_depth=i,random_state=12345)
    DS.fit(features_upsampled_train,target_upsampled_train)
    predictions_ds=DS.predict(features_valid)
    score_ds=f1_score(target_valid, predictions_ds)
    prediction_auc_ds=DS.predict_proba(features_valid)[:,1]
    auc_roc_ds=roc_auc_score(target_valid,prediction_auc_ds)
    print("Глубина дерева:",i)
    print("AUC-ROC:",auc_roc_ds)
    print("F1:",score_ds)

Глубина дерева: 1
AUC-ROC: 0.7206601841329896
F1: 0.4854368932038835
Глубина дерева: 2
AUC-ROC: 0.772215379799808
F1: 0.5066666666666667
Глубина дерева: 3
AUC-ROC: 0.8250668553814329
F1: 0.5066666666666667
Глубина дерева: 4
AUC-ROC: 0.8459400240883511
F1: 0.5555555555555556
Глубина дерева: 5
AUC-ROC: 0.8603011724358494
F1: 0.5394990366088632
Глубина дерева: 6
AUC-ROC: 0.8585319715022559
F1: 0.5576519916142558
Глубина дерева: 7
AUC-ROC: 0.8556842384610673
F1: 0.5650406504065042
Глубина дерева: 8
AUC-ROC: 0.839846487795916
F1: 0.548936170212766
Глубина дерева: 9
AUC-ROC: 0.8201062881483951
F1: 0.5527426160337553


Теперь попробуем метод downsampe,который убирает часть объектов

In [33]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled
features_downsample_train,target_downsample_train=downsample(features_train, target_train, fraction=0.5)


In [34]:
score_ds=0
depth_ds=0
for i in range(1,10):
    DS = DecisionTreeClassifier(max_depth=i,random_state=12345)
    DS.fit(features_downsample_train,target_downsample_train)
    predictions_ds=DS.predict(features_valid)
    score_ds=f1_score(target_valid, predictions_ds)
    prediction_auc_ds=DS.predict_proba(features_valid)[:,1]
    auc_roc_ds=roc_auc_score(target_valid,prediction_auc_ds)
    print("Глубина дерева:",i)
    print("AUC-ROC:",auc_roc_ds)
    print("F1:",score_ds)

Глубина дерева: 1
AUC-ROC: 0.7197347559523405
F1: 0.4927536231884057
Глубина дерева: 2
AUC-ROC: 0.7713307793330113
F1: 0.5337423312883436
Глубина дерева: 3
AUC-ROC: 0.8228553542144408
F1: 0.573134328358209
Глубина дерева: 4
AUC-ROC: 0.8483012268728013
F1: 0.5448504983388704
Глубина дерева: 5
AUC-ROC: 0.8545376601637191
F1: 0.5913978494623656
Глубина дерева: 6
AUC-ROC: 0.8487333201777366
F1: 0.6022727272727273
Глубина дерева: 7
AUC-ROC: 0.8236821154199471
F1: 0.5941644562334218
Глубина дерева: 8
AUC-ROC: 0.8162854945937302
F1: 0.582010582010582
Глубина дерева: 9
AUC-ROC: 0.7860253540103022
F1: 0.5891472868217054


F1 метрика уменьшилась у логистической регрессии до 0.47

In [35]:
LG = LogisticRegression(max_iter=100000,random_state=12345,solver='liblinear')
LG.fit(features_downsample_train,target_downsample_train)
predictions_lg=LG.predict(features_valid)
score_lg=f1_score(target_valid, predictions_lg)
prediction_auc_lg=LG.predict_proba(features_valid)[:,1]
auc_roc_lg=roc_auc_score(target_valid,prediction_auc_lg)
print("F1:",score_lg)
print('AUC-ROC:',auc_roc_lg)

F1: 0.4764705882352941
AUC-ROC: 0.7951945780796004


 F1 метрика у случайного леса увеличилась до 0.61(6 деревьев)

In [36]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_downsample_train, target_downsample_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [5, 7, 9, 11, 13, 15],
                         'max_features': ['log2'],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [37]:
predictions_cv = CV_rfc.best_estimator_.predict(features_valid)
print("F1:", f1_score(predictions_cv, target_valid))

F1: 0.6388059701492537


F1 метрика у решающего дерева уменьшилась до 0.6

Было протестировано два методы по борьбе с дисбалансом в выборке, метод downsample дает лучший результат для случайного леса ( f1 метрика доходит до 0.61), метод upsample показывает лучший результат для логистической регрессии (f1 метрика доходит до 0.49), оба метода плохо работают с решающим деревом, в обоих случаях f1 метрика уменьшилась. Далее проведем тестирование, полученных моеделей.

## Тестирование модели

Качество модели на тестовой выборке уменьшилось (f1 всего 0.38)

In [38]:
#код ревьюера
features_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,Germany,Spain
82,-1.110092,-0.275810,-1.222668,-1.225992,0.802257,0,0,-1.287514,0,0,0
7895,0.182954,0.392879,-1.222668,0.586366,-0.911510,1,1,0.004871,1,0,1
9458,-1.316979,-0.275810,1.123461,1.312961,0.802257,1,0,0.958136,1,0,0
2560,-0.003244,-0.848973,0.453138,-1.225992,-0.911510,0,0,-0.558374,0,0,1
6802,0.120888,-0.657919,1.458622,-1.225992,0.802257,0,0,-0.608881,1,0,0


In [40]:

predictions_test = CV_rfc.best_estimator_.predict(features_test)
print("F1:", f1_score(predictions_test, target_test))

F1: 0.6359223300970873


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*

Итог;
Данные были изучены и подготовлены,выборка разбита на две, признаки масштабированы, все признаки приведены к численным , выделили целевой признак, все пропущенные значение удалены.Были обучены три разные модели: решающее дерево, логистическая регрессия и случайный лес. Лучший результат выдало решающее дерево с глубиной 9,F1=0.63,AUC-ROC=0.84. В наших данных наблюдался дисбаланс с перевесом в сторону отрицательного класса.Было протестировано два методы по борьбе с дисбалансом в выборке, метод downsample дает лучший результат для случайного леса ( f1 метрика доходит до 0.61), метод upsample показывает лучший результат для логистической регрессии (f1 метрика доходит до 0.49), оба метода плохо работают с решающим деревом, в обоих случаях f1 метрика уменьшилась.На тестовой выборке модель логистической регрессии показало результат хуже